In [4]:
from typing import Dict, List, Any

import duckdb
import numpy as np
import pandas as pd
import pyarrow as pa


def set_attr(conn: duckdb.DuckDBPyConnection) -> None:
    conn.execute("SET max_memory='64GB';")
    conn.execute("SET threads=4;")
    conn.execute("SET enable_progress_bar=1;")
    conn.execute("SET temp_directory = 'C:/Projet/tmp/';")
    conn.execute("SET disable_parquet_prefetching = TRUE;")
    conn.execute("SET parquet_metadata_cache = TRUE;")


def decorate_df(df: pd.DataFrame, caption: str):
    return (
        df.style.set_caption(caption)
        .set_table_styles(
            [
                {
                    "selector": "caption",
                    "props": [("caption-side", "top"), ("font-size", "1.25em")],
                },
                {"selector": "th", "props": [("text-align", "center")]},
                {"selector": "th.row_heading", "props": [("text-align", "right")]},
                {
                    "selector": "td:hover",
                    "props": [("background-color", "#ffff99"), ("color", "black")],
                },
                {
                    "selector": "table",
                    "props": [
                        ("border", "1px solid black"),
                        ("border-collapse", "collapse"),
                    ],
                },
                {
                    "selector": "th, td",
                    "props": [("border", "1px solid black")],
                },
            ],
            overwrite=False,
        )
        .format(precision=2, thousands=" ")
        .set_sticky(axis=0)
        .set_sticky(axis=1)
    )


seed = 42

In [13]:
def _build_projection(mapping: Dict[str, Dict[str, Any]]) -> str:
    """Return the SELECT list that converts invalid values to NULL and applies alias/type."""
    parts = []
    for raw, info in mapping.items():
        alias = info["alias"]
        inval = info.get("invalid", [])
        typ = info["type"]

        if not inval:
            parts.append(f"CAST({raw} AS {typ}) AS {alias}")
            continue
        elif not isinstance(inval, (list, tuple)):
            inval = [inval]

        # Proper quoting for string invalids
        in_list = ", ".join(f"'{v}'" if isinstance(v, str) else str(v) for v in inval)
        parts.append(
            f"CAST(CASE WHEN {raw} IN ({in_list}) THEN NULL ELSE {raw} END AS {typ}) AS {alias}"
        )
    return ",\n       ".join(parts)


def extract(
    conn: duckdb.DuckDBPyConnection,
    parquet_path: str,
    mapping: Dict[str, Dict[str, Any]],
    *,
    show=False,
) -> None:
    """Read Parquet, clean invalid values, and create stg.transactions."""
    projection = _build_projection(mapping)

    conn.execute("CREATE SCHEMA IF NOT EXISTS stg")
    conn.execute("DROP TABLE IF EXISTS stg.transactions")

    conn.execute(
        f"""
        CREATE TABLE stg.transactions AS
        SELECT {projection}
        FROM read_parquet('{parquet_path}');
        """
    )

    if show:
        conn.table("stg.transactions").show(max_rows=40)


In [16]:
import json

with open("columns_map.json", "r", encoding="utf-8") as f:
    columns_map = json.load(f)

In [17]:
with duckdb.connect(database="projet.db", read_only=False) as conn:
    set_attr(conn)
    extract(conn, "C:\Projet\Raw_data\A202401.parquet", columns_map, show=True)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────────┬──────────────────────────┬──────────────────┬─────────────────────────┬──────────────────┬─────────────────────────┬───────────────────────────────┬──────────────────────┬─────────────────┬────────────────┬───────────────────────────┬───────────────────────────────┬─────────────────────────┬────────────────┬─────────────────────────┬────────────────────┬────────────────────────────┬───────────────────────────┬─────────────────────┬──────────────────────────┬─────────────────────┬──────────────────────┬──────────────────────────────┬──────────────────────────────┬────────────────────────┬───────────────┬────────────────────────────┬──────────────┬───────────────────────┬─────────────────────┬────────────────────┬────────────────────────┬─────────────────────┬───────────────────────────┬──────────────────────┬──────────────────────────┬────────────────┬───────────────┐
│ prise_en_charge │ tranche_age_bénéficiaire │ nature_assurance │ nature_accident_travail │ top_béné

In [18]:
def exploratoire(
    conn: duckdb.DuckDBPyConnection,
    stg_table: str,
    columns_map: Dict[str, Dict[str, Any]],
    *,
    show: bool = False,
) -> duckdb.DuckDBPyRelation:
    """Compute per‑column null statistics after extract()."""
    conn.execute(
        """
        CREATE OR REPLACE TABLE stg.exploratoire (
            column_name TEXT,
            nb_total BIGINT,
            nb_inutilisables BIGINT,
            pct_inutilisables DOUBLE
        );
        """
    )

    for meta in columns_map.values():
        if not meta.get("invalid"):
            continue
        col = meta["alias"]
        conn.execute(
            f"""
            INSERT INTO stg.exploratoire
            SELECT
                '{col}',
                COUNT(*) AS nb_total,
                COUNT(*) FILTER (WHERE {col} IS NULL) AS nb_inutilisables,
                ROUND(nb_inutilisables / nb_total, 6) AS pct_inutilisables
            FROM {stg_table};
            """
        )

    if show:
        conn.table("stg.exploratoire").show(max_rows=40)
    else:
        return conn.table("stg.exploratoire")

In [19]:
with duckdb.connect(database="projet.db", read_only=False) as conn:
    set_attr(conn)
    exploratoire(conn, "stg.transactions", columns_map, show=True)

┌───────────────────────────────┬──────────┬──────────────────┬───────────────────┐
│          column_name          │ nb_total │ nb_inutilisables │ pct_inutilisables │
│            varchar            │  int64   │      int64       │      double       │
├───────────────────────────────┼──────────┼──────────────────┼───────────────────┤
│ tranche_age_bénéficiaire      │ 38758176 │            40558 │          0.001046 │
│ nature_assurance              │ 38758176 │                1 │               0.0 │
│ nature_accident_travail       │ 38758176 │         38745854 │          0.999682 │
│ top_bénéficiaire              │ 38758176 │         34312781 │          0.885304 │
│ qualité_du_bénéficiaire       │ 38758176 │                7 │               0.0 │
│ région_résidence_bénéficiaire │ 38758176 │          2427121 │          0.062622 │
│ sexe_du_bénéficiaire          │ 38758176 │                0 │               0.0 │
│ complément_acte               │ 38758176 │                0 │             

In [20]:
def impute_nulls_with_mode(
    conn: duckdb.DuckDBPyConnection, table: str, columns: List[str]
) -> None:
    for col in columns:
        mode_query = (
            f"SELECT mode({col}) AS mode_val FROM {table} WHERE {col} IS NOT NULL"
        )
        mode_val = conn.execute(mode_query).fetchone()[0]

        if mode_val is not None:
            update_query = f"""
                UPDATE {table}
                SET {col} = {mode_val}
                WHERE {col} IS NULL
            """
            conn.execute(update_query)


def add_index(
    conn: duckdb.DuckDBPyConnection, table: str, index_name: str, seed: int = 42
) -> None:
    import math

    conn.execute(f"ALTER TABLE {table} ADD COLUMN IF NOT EXISTS {index_name} UTINYINT")

    conn.execute(f"SELECT SETSEED({math.tanh(seed)})")

    conn.execute(f"""
    UPDATE {table}
    SET {index_name} = floor(random()*10)::UTINYINT
    WHERE {index_name} IS NULL
""")


def feature_engineering(
    conn: duckdb.DuckDBPyConnection,
    X_cols: List[str],
    Y_cols: List[str],
    columns_map: Dict[str, Dict[str, Any]],
    from_table: str,
    to_table: str,
    new_features: Dict[str, List[str]],
    index_name: str,
) -> None:
    for _, meta in columns_map.items():
        alias = meta["alias"]
        category = meta["category"]
        if alias in X_cols and category in new_features.keys():
            new_features[category].append(alias)

    concat_sql_parts = []
    for feature, value in new_features.items():
        if value:
            concat_sql_parts.append(f"CONCAT_WS('_', {', '.join(value)}) AS {feature}")

    conn.execute(f"""
        CREATE OR REPLACE TABLE {to_table} AS 
        SELECT {",".join(concat_sql_parts + Y_cols)}, {index_name}
        FROM {from_table}""")


In [29]:
threshold = 0.2
stg_table = "stg.transactions"
concat_table = "stg.transactions_concated"
index_name = "index"

with duckdb.connect(database="projet.db", read_only=False) as conn:
    set_attr(conn)
    extract(conn, "C:\Projet\Raw_data\A202401.parquet", columns_map)

    X_cols = (
        exploratoire(conn, stg_table, columns_map)
        .filter(f"pct_inutilisables < {threshold}")
        .project("column_name")
        .to_df()["column_name"]
        .tolist()
    )
    Y_cols = ["prise_en_charge"]

    impute_nulls_with_mode(conn, stg_table, X_cols)
    add_index(conn, stg_table, index_name)
    """
    new_features = {
        "BENEFICIAIRE": [],
        "PRESTATION": [],
        "EXECUTANT": [],
        "PRESCRIPTEUR": [],
        "ORGANISME": [],
        "PERIODE": [],
    }

    feature_engineering(
        conn,
        X_cols,
        Y_cols,
        columns_map,
        stg_table,
        concat_table,
        new_features,
        index_name,
    )

    X_cols = list(new_features.keys())"""

    X = (
        conn.table(stg_table)
        .filter(f"{index_name} = 0")
        .select(*X_cols)
        .fetch_arrow_table()
        .to_pandas(split_blocks=True)
    )
    X = X.astype({col: "category" for col in X_cols})
    y = (
        conn.table(stg_table)
        .filter(f"{index_name} = 0")
        .select(*Y_cols)
        .fetch_arrow_table()
        .to_pandas()
    ).squeeze()

    print(X.shape, y.shape)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(3875590, 16) (3875590,)


In [30]:
def plot_mutual_info(X, y, *, discrete_features=True, random_state=42, n_jobs=-1):
    from sklearn.feature_selection import mutual_info_classif

    mi = mutual_info_classif(
        X,
        y,
        discrete_features=discrete_features,
        random_state=random_state,
        n_jobs=n_jobs,
    )

    mi_df = pd.DataFrame({"feature": X_cols, "mutual_info": mi})

    mi_df = mi_df.sort_values(by="mutual_info", ascending=False).reset_index(drop=True)

    display(decorate_df(mi_df, "Mutual Information"))


plot_mutual_info(X, y)

,feature,mutual_info
0,complément_acte,0.02
1,nature_assurance,0.01
2,code_qualificatif_parcours,0.01
3,catégorie_exécutant,0.00
4,catégorie_du_prescripteur,0.00
5,tranche_age_bénéficiaire,0.00
6,mois_de_soins,0.00
7,année_de_soins,0.00
8,région_résidence_bénéficiaire,0.00
9,région_ps_exécutant,0.00


In [25]:
import pprint

from boruta import BorutaPy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted


class BorutaFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, estimator, output=False):
        self.estimator = estimator
        self.output = output

    def fit(self, X, y):
        self.feature_names_ = X.columns.tolist()
        self.selector = BorutaPy(
            estimator=self.estimator,
            n_estimators="auto",
            max_iter=10,
            random_state=seed,
        )
        self.selector.fit(X.values, y)
        self._sklearn_is_fitted = True
        if self.output:
            self._show_ranking()

        return self

    def transform(self, X):
        check_is_fitted(self, attributes=["selector"])
        return X.iloc[:, self.selector.support_]

    def _show_ranking(self):
        print("------Support and Ranking for each feature------")
        feature_info = [
            (self.selector.support_[i], self.selector.ranking_[i], feature)
            for i, feature in enumerate(self.feature_names_)
        ]
        feature_info.sort(key=lambda x: (x[1], x[2]))
        digit = len(str(len(self.feature_names_)))

        for support, ranking, feature in feature_info:
            status = "✅" if support else "❌"
            print(f"{status} Rank: {ranking:<{digit}} - {feature}")

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

with duckdb.connect(database="projet.db", read_only=False) as conn:
    set_attr(conn)
    for i in range(5):
        query = f"""
            SELECT {", ".join(X_cols + Y_cols)}
            FROM {stg_table}
            USING SAMPLE 100000 ROWS
        """
        query_result = (
            conn.execute(query).fetch_arrow_table().to_pandas(split_blocks=True)
        )

        X_tiny = query_result[X_cols].astype({col: "category" for col in X_cols})
        y_tiny = query_result[Y_cols].squeeze()

        X_train_tiny, X_test_tiny, y_train_tiny, y_test_tiny = train_test_split(
            X_tiny, y_tiny, test_size=0.2, random_state=seed
        )

        # Boruta Feature Selection
        rf = BorutaFeatureSelector(
            estimator=ExtraTreesClassifier(n_jobs=-1),
            output=True,
        )
        rf.fit(X_train_tiny, y_train_tiny)

------Support and Ranking for each feature------
❌ Rank: 2  - tranche_age_bénéficiaire
❌ Rank: 3  - région_résidence_bénéficiaire
❌ Rank: 4  - région_organisme_liquidation
❌ Rank: 5  - région_ps_exécutant
❌ Rank: 6  - catégorie_exécutant
❌ Rank: 7  - mois_de_soins
❌ Rank: 8  - catégorie_du_prescripteur
❌ Rank: 9  - code_qualificatif_parcours
❌ Rank: 10 - complément_acte
❌ Rank: 10 - nature_assurance
❌ Rank: 12 - sexe_du_bénéficiaire
❌ Rank: 13 - année_de_soins
❌ Rank: 14 - qualité_du_bénéficiaire
❌ Rank: 15 - statut_juridique_ps
❌ Rank: 16 - code_secteur
❌ Rank: 17 - type_prise_en
------Support and Ranking for each feature------
❌ Rank: 2  - tranche_age_bénéficiaire
❌ Rank: 3  - région_résidence_bénéficiaire
❌ Rank: 4  - région_organisme_liquidation
❌ Rank: 5  - région_ps_exécutant
❌ Rank: 6  - catégorie_exécutant
❌ Rank: 7  - mois_de_soins
❌ Rank: 8  - catégorie_du_prescripteur
❌ Rank: 9  - code_qualificatif_parcours
❌ Rank: 10 - sexe_du_bénéficiaire
❌ Rank: 11 - complément_acte
❌ Ran

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor

with duckdb.connect(database="projet.db", read_only=False) as conn:
    set_attr(conn)
    for i in range(5):
        query = f"""
            SELECT {", ".join(X_cols + Y_cols)}
            FROM {concat_table}
            USING SAMPLE 100000 ROWS
        """
        query_result = (
            conn.execute(query).fetch_arrow_table().to_pandas(split_blocks=True)
        )

        X_tiny = query_result[X_cols].astype({col: "category" for col in X_cols})
        Y_tiny = query_result[Y_cols]
        y_tiny = Y_tiny["mnt_pay"]

        X_train_tiny, X_test_tiny, y_train_tiny, y_test_tiny = train_test_split(
            X_tiny, y_tiny, test_size=0.2, random_state=seed
        )

        # Boruta Feature Selection
        rf = BorutaFeatureSelector(
            estimator=ExtraTreesRegressor(n_jobs=-1),
            output=True,
        )
        rf.fit(X_train_tiny, y_train_tiny)


------Support and Ranking for each feature------
❌ Rank: 2 - BENEFICIAIRE
❌ Rank: 3 - EXECUTANT
❌ Rank: 4 - ORGANISME
❌ Rank: 6 - PERIODE
❌ Rank: 6 - PRESCRIPTEUR
❌ Rank: 6 - PRESTATION
------Support and Ranking for each feature------
❌ Rank: 2 - BENEFICIAIRE
❌ Rank: 3 - EXECUTANT
❌ Rank: 4 - PRESTATION
❌ Rank: 5 - PERIODE
❌ Rank: 6 - ORGANISME
❌ Rank: 6 - PRESCRIPTEUR
------Support and Ranking for each feature------
❌ Rank: 2 - BENEFICIAIRE
❌ Rank: 3 - EXECUTANT
❌ Rank: 4 - ORGANISME
❌ Rank: 5 - PRESTATION
❌ Rank: 6 - PRESCRIPTEUR
❌ Rank: 7 - PERIODE
------Support and Ranking for each feature------
❌ Rank: 2 - BENEFICIAIRE
❌ Rank: 3 - EXECUTANT
❌ Rank: 4 - PRESCRIPTEUR
❌ Rank: 5 - ORGANISME
❌ Rank: 6 - PERIODE
❌ Rank: 7 - PRESTATION
------Support and Ranking for each feature------
❌ Rank: 2 - BENEFICIAIRE
❌ Rank: 3 - ORGANISME
❌ Rank: 4 - PRESTATION
❌ Rank: 5 - EXECUTANT
❌ Rank: 6 - PERIODE
❌ Rank: 7 - PRESCRIPTEUR


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV
from tabulate import tabulate


def optimize_hyperparameters(
    X,
    y,
    estimator,
    param_distributions,
    *,
    scoring="accuracy",
    n_jobs=-1,
    cv=5,
    random_state=42,
) -> RandomizedSearchCV:
    """Optimize hyperparameters using RandomizedSearchCV."""
    randomized_search = RandomizedSearchCV(
        estimator=estimator,
        param_distributions=param_distributions,
        scoring=scoring,
        n_jobs=n_jobs,
        cv=cv,
        return_train_score=True,
        random_state=random_state,
    )
    randomized_search.fit(X, y)
    return randomized_search.best_estimator_, randomized_search.best_params_


def regression_report(pipelines, X_train, X_test, y_train, y_test, printer="styler"):
    metrics = ["MSE", "RMSE", "MAE", "R2"]
    reports = []
    for name, model in pipelines.items():
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_results = [
            mean_squared_error(y_train, y_train_pred),
            np.sqrt(mean_squared_error(y_train, y_train_pred)),
            mean_absolute_error(y_train, y_train_pred),
            r2_score(y_train, y_train_pred),
        ]

        test_results = [
            mean_squared_error(y_test, y_test_pred),
            np.sqrt(mean_squared_error(y_test, y_test_pred)),
            mean_absolute_error(y_test, y_test_pred),
            r2_score(y_test, y_test_pred),
        ]

        reports.append(
            pd.DataFrame(
                {(name, "Train"): train_results, (name, "Test"): test_results},
                index=metrics,
            )
        )

    if printer == "styler":
        display(decorate_df(pd.concat(reports, axis=1), "Regression Report"))
    elif printer == "tabulate":
        # Tabulate doesn't support MultiIndex
        print(
            f"=== Regression Report: {model.__class__.__name__} ===\n",
            tabulate(
                pd.concat(reports, axis=1),
                headers="keys",
                tablefmt="fancy_grid",
                stralign="center",
                floatfmt=".2f",
            ),
            sep="",
        )
    else:
        raise ValueError("Printer must be 'df' or 'tabulate'")

In [ ]:
from lightgbm import LGBMRegressor
from scipy.stats import randint, uniform

param_distributions = {
    "model__num_leaves": randint(20, 150),  # Controls tree complexity
    "model__max_depth": randint(3, 15),  # Limits tree depth
    "model__learning_rate": uniform(0.01, 0.3),  # Step size shrinkage
    "model__n_estimators": randint(100, 1000),  # Number of boosting rounds
    "model__min_child_samples": randint(5, 100),  # Minimum data in one leaf
    "model__subsample": uniform(0.5, 0.5),  # Subsample ratio of the training instance
    "model__colsample_bytree": uniform(
        0.5, 0.5
    ),  # Subsample ratio of columns when constructing each tree
    "model__reg_alpha": uniform(0.0, 1.0),  # L1 regularization term
    "model__reg_lambda": uniform(0.0, 1.0),  # L2 regularization term
}

model = LGBMRegressor(random_state=seed)

y = Y.iloc[:, 2]

model.fit(X, y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed
)

pipeline, _ = optimize_hyperparameters(
    X_train,
    y_train,
    model,
    param_distributions,
    scoring="r2",
    n_jobs=-1,
    cv=5,
    random_state=seed,
)

pipelines = {
    "LGBMRegressor": pipeline,
}

regression_report(
    pipelines,
    X_train,
    X_test,
    y_train,
    y_test,
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.099442 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 3875165, number of used features: 11
[LightGBM] [Info] Start training from score 337.348045
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.078573 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 88
[LightGBM] [Info] Number of data points in the train set: 3100132, number of used features: 11
[LightGBM] [Info] Start training from score 334.406316
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp